In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

# 알림 처리 함수
def handle_alert(driver):
    try:
        alert = WebDriverWait(driver, 5).until(EC.alert_is_present())
        alert_text = alert.text
        print(f"알림: {alert_text}")
        alert.accept()  # 알림 수락
        print("알림을 처리했습니다.")
    except Exception as e:
        print("알림 처리 실패:", e)

# 다운로드 완료 여부를 확인하는 함수
def is_download_complete(download_dir):
    files_before = set(os.listdir(download_dir))
    time.sleep(5)  # 다운로드 진행 중 잠시 대기
    files_after = set(os.listdir(download_dir))
    return len(files_before) < len(files_after)

# 다운로드와 관련된 동작을 처리하는 함수
def perform_download_actions(driver, download_dir):
    try:
        # 전체 선택 체크박스 클릭
        WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, "checkAll"))
        ).click()

        # 다운로드 버튼 클릭 (이벤트 전파 막기)
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "btn_download"))
        )
        driver.execute_script("""  
            var element = arguments[0];
            element.addEventListener('click', function(event) {
                event.stopPropagation();  // 이벤트 전파 막기
            });
            arguments[0].click();
        """, download_button)

        # 다운로드 완료 여부 확인
        if is_download_complete(download_dir):
            print("다운로드 완료")

        # 라디오 버튼 클릭
        purpose_radio = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='radio'][name='reqstPurposeCd'][value='F00408']"))
        )
        driver.execute_script("arguments[0].click();", purpose_radio)

        # 신청 버튼 클릭 (이벤트 전파 막기)
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "btn_request"))
        )

        # 신청 버튼 클릭 시 이벤트 전파 막기
        driver.execute_script("""  
            var element = arguments[0];
            element.addEventListener('click', function(event) {
                event.stopPropagation();  // 이벤트 전파 막기
            });
            arguments[0].click();
        """, submit_button)

    except Exception as e:
        print(f"다운로드 관련 작업에서 오류 발생: {e}")

# Chrome WebDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 페이지 접속
url = "https://data.kma.go.kr/data/grnd/selectAwosRltmList.do?pgmNo=638&tabNo=1"
driver.get(url)

# 로그인 정보
login_id = ""  # 아이디 입력
password = ""  # 비밀번호 입력

try:
    # 로그인
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "loginBtn"))
    )
    login_button.click()

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "loginId")))

    driver.find_element(By.ID, "loginId").send_keys(login_id)
    driver.find_element(By.ID, "passwordNo").send_keys(password)
    driver.find_element(By.ID, "loginbtn").click()

    # 팝업 닫기
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "loginPopCloseBtn"))
    ).click()

    # 기관 선택
    institution_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "stnGrpSn"))
    )
    Select(institution_select).select_by_value("985")

    # 지점 선택
    stn_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "btnStn2"))
    )
    stn_button.click()
    WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "wrap_lst_chk"))
    )
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='전체선택']"))
    ).click()
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='선택완료']"))
    ).click()

    # 조회 버튼 클릭
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='조 회']"))
    ).click()

    # 페이지 순회 및 다운로드
    total_pages = 10  # 반복할 페이지 수 설정
    download_dir = "C:/Users/uos/Downloads"  # 다운로드 폴더 경로 설정

    for current_page in range(1, total_pages + 1):
        page_number = current_page
        print(f"다음 페이지로 이동 시도: 페이지 {page_number}")
        perform_download_actions(driver, download_dir)
        
        try:
            # 페이지 이동 스크립트 실행
            driver.execute_script(f"goPage({page_number+1}); return false;")
            # 페이지가 로드될 때까지 기다리기
            WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element((By.CLASS_NAME, "on"), str(page_number+1))
            )
            print(f"다음 페이지로 이동 성공: 페이지 {page_number+1}")
        except Exception as e:
            print(f"페이지 이동 실패: {e}, 다시 시도합니다...")
            time.sleep(2)  # 잠시 대기 후 재시도
            continue  # 페이지 이동 실패 시 다시 시도

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    time.sleep(5)
    driver.quit()
